In [1]:
%pwd
%cd /Users/ashwin/Desktop/College Work/Fourth Year/L48/GroupProject/tti

/Users/ashwin/Desktop/College Work/Fourth Year/L48/GroupProject/tti


In [2]:
import os

import numpy as np
import pandas as pd
from tqdm.notebook import trange
from tqdm import tqdm
import importlib

import tti_explorer
from tti_explorer import config, utils
from tti_explorer.case import simulate_case, CaseFactors
from tti_explorer.contacts import EmpiricalContactsSimulator
from tti_explorer.strategies import TTIFlowModel, RETURN_KEYS

importlib.reload(tti_explorer)
importlib.reload(tti_explorer.config)
importlib.reload(tti_explorer.utils)
importlib.reload(tti_explorer.case)
importlib.reload(tti_explorer.strategies)
importlib.reload(tti_explorer.contacts)

<module 'tti_explorer.contacts' from '/Users/ashwin/Desktop/College Work/Fourth Year/L48/GroupProject/tti/tti_explorer/contacts.py'>

In [3]:
def print_doc(func):
    print(func.__doc__)

In [4]:
name = 'S3_test_based_TTI'
rng = np.random.RandomState(0)
case_config = config.get_case_config("delve_mostly_uni")
#case_config = config.get_case_config("delve_mostly_old")
#case_config = config.get_case_config("delve_mostly_twenties")
contacts_config = config.get_contacts_config("delve")
policy_config = config.get_strategy_configs("delve", name)[name]
factor_config = utils.get_sub_dictionary(policy_config, config.DELVE_CASE_FACTOR_KEYS)
strategy_config = utils.get_sub_dictionary(policy_config, config.DELVE_STRATEGY_FACTOR_KEYS)

In [5]:
def load_csv(pth):
    return np.loadtxt(pth, dtype=int, skiprows=1, delimiter=",")

In [6]:
path_to_bbc_data = os.path.join("data", "processed")

child_no_school = load_csv(os.path.join(path_to_bbc_data, "childnoschool.csv"))
child_school = load_csv(os.path.join(path_to_bbc_data, "childschool.csv"))
university = load_csv(os.path.join(path_to_bbc_data, "university.csv"))
twenties = load_csv(os.path.join(path_to_bbc_data, "twenties.csv"))
thirties_to_fifties = load_csv(os.path.join(path_to_bbc_data, "thirtiestofifties.csv"))
fifties_to_seventies = load_csv(os.path.join(path_to_bbc_data, "fiftiestoseventies.csv"))
seventy_plus = load_csv(os.path.join(path_to_bbc_data, "seventyplus.csv"))

In [7]:
double_dose = True
vaccine_strategy = 'young'
#Vaccine strategies are: none, all, gov, young_inc_children, 
#                        young_exc_children

In [8]:
simulate_contacts = EmpiricalContactsSimulator(child_no_school, child_school, university, twenties, thirties_to_fifties, fifties_to_seventies, seventy_plus, double_dose, vaccine_strategy, rng)
tti_model = TTIFlowModel(rng, **strategy_config)

In [9]:
n_seperate_starting_cases = 1000
n_repetitions = 10
outputs = list()
death_rates = []

for i in tqdm(range(n_seperate_starting_cases)):
    old_probs = []
    for i in range(n_repetitions):
        case = simulate_case(rng, p_for_categories_continued=old_probs, **case_config)
        case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
        (contacts, old_probs, death_rate) = simulate_contacts(case, double_dose, vaccine_strategy, **contacts_config)
        res = tti_model(case, contacts, case_factors)
        outputs.append(res)
        death_rates.append(death_rate)

100%|██████████| 1000/1000 [00:13<00:00, 73.06it/s]


In [10]:
to_show = [
    RETURN_KEYS.base_r,
    RETURN_KEYS.reduced_r
]

x = pd.DataFrame(outputs).mean(0).loc[to_show].to_frame().to_numpy().flatten()
base_r = x[0]
effective_r = x[1]
relevant_death_rate_sum = sum(n for n in death_rates if n != -1)
relevant_death_rate_count = sum(1 for n in death_rates if n != -1)
dr = relevant_death_rate_sum / relevant_death_rate_count

print("Average Mortality Rate: " + str(round(dr*100,2)) + "%")
print("Base R: " + str(round(base_r, 3)))
print("Effective R: " + str(round(effective_r, 3)))

Average Mortality Rate: 2.52%
Base R: 3.901
Effective R: 1.296
